In [1]:
import os
import numpy as np
import random
from collections import Counter
from PIL import Image
# from google.colab import files 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, Model, clone_model      
from keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Input, ConvLSTM2D,MaxPooling3D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, EfficientNetB2
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Input
from tensorflow.keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import shutil

#  Connectivity

In [43]:
data_dir = 'F:/Abdipour/Connectivity/Pearson images'

def extract_numeric_part(folder_name):
    parts = folder_name.split('_')
    for part in parts:
        if part.startswith('subject'):
            return int(part.replace('subject', ''))
    return -1  # Return -1 if no numeric part is found

folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

subject_folders, labels = [], []

for folder_name in folders:
    subject_num = extract_numeric_part(folder_name)
    subject_folder = os.path.join(data_dir, folder_name)
    subject_folders.append(subject_folder)

    if subject_num <= 36:
        labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        labels.append(1)  # HC group

subject_folders = np.array(subject_folders)
labels = np.array(labels)

print("Subject Folders:", subject_folders)
print("Labels:", labels)

Subject Folders: ['F:/Abdipour/Connectivity/Pearson images\\Subject1'
 'F:/Abdipour/Connectivity/Pearson images\\Subject10'
 'F:/Abdipour/Connectivity/Pearson images\\Subject11'
 'F:/Abdipour/Connectivity/Pearson images\\Subject12'
 'F:/Abdipour/Connectivity/Pearson images\\Subject13'
 'F:/Abdipour/Connectivity/Pearson images\\Subject14'
 'F:/Abdipour/Connectivity/Pearson images\\Subject15'
 'F:/Abdipour/Connectivity/Pearson images\\Subject16'
 'F:/Abdipour/Connectivity/Pearson images\\Subject17'
 'F:/Abdipour/Connectivity/Pearson images\\Subject18'
 'F:/Abdipour/Connectivity/Pearson images\\Subject19'
 'F:/Abdipour/Connectivity/Pearson images\\Subject2'
 'F:/Abdipour/Connectivity/Pearson images\\Subject20'
 'F:/Abdipour/Connectivity/Pearson images\\Subject21'
 'F:/Abdipour/Connectivity/Pearson images\\Subject22'
 'F:/Abdipour/Connectivity/Pearson images\\Subject23'
 'F:/Abdipour/Connectivity/Pearson images\\Subject24'
 'F:/Abdipour/Connectivity/Pearson images\\Subject25'
 'F:/Abdipour

In [30]:
def load_images_and_labels(subjects, labels):
    images, labels_t = [], []
    for subject_folder, label in zip(subjects, labels):
        for filename in os.listdir(subject_folder):
            img_path = os.path.join(subject_folder, filename)
            try:
                img = Image.open(img_path)
                img = img.resize((224, 224))
                img_array = np.array(img, dtype=np.float32)
                img_array = img_array / 255.0  
                images.append(img_array)
                labels_t.append(label)
            except Exception as e:
                print(f"Error loading image: {img_path}, {e}")

    return np.array(images), np.array(labels_t)

In [31]:
def load_and_preprocess_data(subject_folders, labels):
    train_subjects, test_subjects, train_labels, test_labels = train_test_split(subject_folders, labels, test_size=0.2, stratify=labels)

    train_subjects, val_subjects, train_labels, val_labels = train_test_split(train_subjects, train_labels, test_size=0.1, stratify=train_labels)

    train_images, train_labels_t = load_images_and_labels(train_subjects, train_labels)
    val_images, val_labels_t = load_images_and_labels(val_subjects, val_labels)

    test_image_counts_per_subject = [len(os.listdir(folder)) for folder in test_subjects]

    test_images, test_labels_t = load_images_and_labels(test_subjects, test_labels)

    return train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject



In [32]:
def create_new_model(base_model):
    model = clone_model(base_model)

    for layer in model.layers:
        layer.trainable = False

    x = model.output
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Dropout(0.2)(x)
    x = Dense(256)(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = Dropout(0.2)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=model.input, outputs=predictions)
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [33]:
def train_and_evaluate_model(base_model, subject_folders, labels, num_epochs=50, num_iterations=5, model_name='model'):
    accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}:")

        model = create_new_model(base_model)

        train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject = load_and_preprocess_data(subject_folders, labels)

        train_data = list(zip(train_images, train_labels_t))
        np.random.shuffle(train_data)
        train_images, train_labels_t = zip(*train_data)

        val_data = list(zip(val_images, val_labels_t))
        np.random.shuffle(val_data)
        val_images, val_labels_t = zip(*val_data)

        # test_data = list(zip(test_images, test_labels_t))
        # np.random.shuffle(test_data)
        # test_images, test_labels_t = zip(*test_data)

        train_images, train_labels_t = np.array(train_images), np.array(train_labels_t)
        val_images, val_labels_t = np.array(val_images), np.array(val_labels_t)
        # test_images, test_labels_t = np.array(test_images), np.array(test_labels_t)



        # checkpoint_filepath = "best_weights_iteration_{iteration + 1}.h5"
        # checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
        #                                        monitor='val_accuracy',
        #                                        mode='max',
        #                                        save_best_only=True,
        #                                        save_weights_only=True,
        #                                        verbose=1)

        # history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=8, validation_data=(val_images, val_labels_t), callbacks=[checkpoint], verbose=1)

        checkpoint_dir = 'F:/Abdipour/Spectrogram Images/'
        os.makedirs(checkpoint_dir, exist_ok=True)

        # local_checkpoint_filepath = os.path.join(checkpoint_dir, f"{model_name}_best_weights_iteration_{iteration + 1}.h5")
        # drive_checkpoint_filepath = f'/content/drive/My Drive/{model_name}_best_weights_iteration_{iteration + 1}.h5'
        local_checkpoint_filepath = checkpoint_dir + '\\' + f"{model_name}_best_weights_iteration_{iteration +1}.weights.h5"

        checkpoint_callback = ModelCheckpoint(filepath=local_checkpoint_filepath,
                                               monitor='val_accuracy',
                                               mode='max',
                                               save_best_only=True,
                                               save_weights_only=True,
                                               verbose=1)

        # class CustomEarlyStopping(EarlyStopping):
        #     def on_epoch_end(self, epoch, logs=None):
        #         if epoch > 30:
        #             super().on_epoch_end(epoch - 30, logs)

        class LearningRateLogger(Callback):
            def on_epoch_end(self, epoch, logs=None):
                logs = logs or {}
                logs['learning_rate'] = self.model.optimizer.learning_rate.numpy()
        lr_logger = LearningRateLogger()
        # initial_epochs = 30        
        # history = model.fit(train_images, train_labels_t, epochs=initial_epochs, batch_size=16, validation_data=(val_images, val_labels_t), verbose=1)

        # early_stopping = CustomEarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', baseline=None, restore_best_weights=False, start_from_epoch=30)
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', baseline=None, restore_best_weights=False)
        # history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=16, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping], initial_epoch=initial_epochs, verbose=1)
        history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=8, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping], verbose=1)

        # if os.path.exists(local_checkpoint_filepath):
        #     print(f"Best weights saved locally at: {local_checkpoint_filepath}")

        #     shutil.copy(local_checkpoint_filepath, drive_checkpoint_filepath)
        #     if os.path.exists(drive_checkpoint_filepath):
        #         print(f"Best weights also copied to Google Drive at: {drive_checkpoint_filepath}")
        #     else:
        #         print(f"Failed to copy best weights to Google Drive at: {drive_checkpoint_filepath}")
        # else:
        #     print(f"Failed to save best weights locally at: {local_checkpoint_filepath}")

        model.load_weights(local_checkpoint_filepath)

        test_predictions = model.predict(test_images)
        test_predictions = (test_predictions > 0.5).astype(int)

        accuracy = accuracy_score(test_labels_t, test_predictions)
        precision = precision_score(test_labels_t, test_predictions)
        recall = recall_score(test_labels_t, test_predictions)
        f1 = f1_score(test_labels_t, test_predictions)

        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        print("ModelCheckpoint callback triggered and saved the best weights.")


        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_accuracy_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        # files.download(file_name)



        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_loss_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        # files.download(file_name)


        cm = confusion_matrix(test_labels_t, test_predictions)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=['AD', 'HC'], yticklabels=['AD', 'HC'])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix - {model_name} - Iteration {iteration + 1}')
        file_name = f'{model_name}_confusion_matrix_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        # files.download(file_name)




        # print("\nPredictions for Each Subject's Images:")
        # start_idx = 0
        # for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
        #     end_idx = start_idx + num_images
        #     subject_images = test_images[start_idx:end_idx]
        #     subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
        #     print(f"Subject {i + 1} ({subject_folder}):")
        #     for j, prediction in enumerate(subject_predictions):
        #         if prediction == 0:
        #             print(f"Image {j + 1}: AD")
        #         else:
        #             print(f"Image {j + 1}: HC")
        #     start_idx = end_idx


        print("\nPredictions for Each Subject's Images:")
        start_idx = 0
        for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
            end_idx = start_idx + num_images
            subject_images = test_images[start_idx:end_idx]
            subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
            ad_count = sum(subject_predictions == 0)
            hc_count = sum(subject_predictions == 1)
            print(f"Subject {i + 1} ({subject_folder}):")
            print(f"AD count: {ad_count}, HC count: {hc_count}")
            start_idx = end_idx




    print("\nAverage Metrics:")
    print(f"Average Accuracy: {np.mean(accuracy_list):.4f}")
    print(f"Average Precision: {np.mean(precision_list):.4f}")
    print(f"Average Recall: {np.mean(recall_list):.4f}")
    print(f"Average F1 Score: {np.mean(f1_list):.4f}")



# ImCoh 

## VGG16

In [15]:
vgg16_base_model = VGG16(weights ='imagenet', include_top =False, input_shape = (224,224,3))

In [16]:
model_name = 'vgg16_imcoh'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - accuracy: 0.9934 - loss: 0.0491
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to F:/Abdipour/Spectrogram Images/\vgg16_imcoh_best_weights_iteration_1.weights.h5
227/227 ━━━━━━━━━━━━━━━━━━━━ 108s 469ms/step - accuracy: 0.9934 - loss: 0.0489 - val_accuracy: 1.0000 - val_loss: 1.5575e-18
Epoch 2/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step - accuracy: 1.0000 - loss: 3.7235e-16
Epoch 2: val_accuracy did not improve from 1.00000
227/227 ━━━━━━━━━━━━━━━━━━━━ 108s 474ms/step - accuracy: 1.0000 - loss: 3.7362e-16 - val_accuracy: 1.0000 - val_loss: 1.5575e-18
Epoch 3/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step - accuracy: 1.0000 - loss: 1.5120e-15
Epoch 3: val_accuracy did not improve from 1.00000
227/227 ━━━━━━━━━━━━━━━━━━━━ 108s 474ms/step - accuracy: 1.0000 - loss: 1.5114e-15 - val_accuracy: 1.0000 - val_loss: 1.5575e-18
Epoch 4/100
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step - accuracy: 1.0000 - l

f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-package


Predictions for Each Subject's Images:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 1 (F:/Abdipour/Connectivity/ImCoh images\Subject3):
AD count: [27], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 487ms/step
Subject 2 (F:/Abdipour/Connectivity/ImCoh images\Subject59):
AD count: [42], HC count: [0]
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 952ms/step
Subject 3 (F:/Abdipour/Connectivity/ImCoh images\Subject15):
AD count: [74], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step
Subject 4 (F:/Abdipour/Connectivity/ImCoh images\Subject53):
AD count: [34], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 666ms/step
Subject 5 (F:/Abdipour/Connectivity/ImCoh images\Subject16):
AD count: [46], HC count: [0]
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 933ms/step
Subject 6 (F:/Abdipour/Connectivity/ImCoh images\Subject10):
AD count: [73], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 7 (F:/Abdipour/Connectivity/ImCoh images\Subject40):
AD count: [31], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 863ms/step
Subject 8 (F:/Abdi

In [17]:
model_name = 'vgg16_imcoh'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 0.9812 - loss: 0.0413
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to F:/Abdipour/Spectrogram Images/\vgg16_imcoh_best_weights_iteration_1.weights.h5
242/242 ━━━━━━━━━━━━━━━━━━━━ 116s 476ms/step - accuracy: 0.9813 - loss: 0.0411 - val_accuracy: 1.0000 - val_loss: 2.3546e-22
Epoch 2/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step - accuracy: 1.0000 - loss: 2.0776e-17
Epoch 2: val_accuracy did not improve from 1.00000
242/242 ━━━━━━━━━━━━━━━━━━━━ 110s 455ms/step - accuracy: 1.0000 - loss: 2.0739e-17 - val_accuracy: 1.0000 - val_loss: 2.3546e-22
Epoch 3/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step - accuracy: 1.0000 - loss: 5.0874e-17
Epoch 3: val_accuracy did not improve from 1.00000
242/242 ━━━━━━━━━━━━━━━━━━━━ 110s 455ms/step - accuracy: 1.0000 - loss: 5.0749e-17 - val_accuracy: 1.0000 - val_loss: 2.3546e-22
Epoch 4/100
242/242 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step - accuracy: 1.0000 - l

f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-package


Predictions for Each Subject's Images:
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step
Subject 1 (F:/Abdipour/Connectivity/ImCoh images\Subject5):
AD count: [36], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 2 (F:/Abdipour/Connectivity/ImCoh images\Subject35):
AD count: [22], HC count: [0]
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 752ms/step
Subject 3 (F:/Abdipour/Connectivity/ImCoh images\Subject36):
AD count: [66], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 4 (F:/Abdipour/Connectivity/ImCoh images\Subject40):
AD count: [31], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 908ms/step
Subject 5 (F:/Abdipour/Connectivity/ImCoh images\Subject25):
AD count: [19], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 256ms/step
Subject 6 (F:/Abdipour/Connectivity/ImCoh images\Subject27):
AD count: [37], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 721ms/step
Subject 7 (F:/Abdipour/Connectivity/ImCoh images\Subject14):
AD count: [16], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 8 (F:/Abdipou

## ResNet50


In [41]:

ResNet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [42]:
model_name = 'ResNet50_imcoh'
train_and_evaluate_model(ResNet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)



Iteration 1:
Epoch 1/100
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step - accuracy: 0.9829 - loss: 0.0326
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to F:/Abdipour/Spectrogram Images/\ResNet50_imcoh_best_weights_iteration_1.weights.h5
243/243 ━━━━━━━━━━━━━━━━━━━━ 121s 477ms/step - accuracy: 0.9829 - loss: 0.0325 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/100
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 2: val_accuracy did not improve from 1.00000
243/243 ━━━━━━━━━━━━━━━━━━━━ 112s 461ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/100
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3: val_accuracy did not improve from 1.00000
243/243 ━━━━━━━━━━━━━━━━━━━━ 112s 461ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/100
243/243 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 1.0000 

f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-package


Predictions for Each Subject's Images:
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
Subject 1 (F:/Abdipour/Connectivity/ImCoh images\Subject32):
AD count: [57], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 437ms/step
Subject 2 (F:/Abdipour/Connectivity/ImCoh images\Subject52):
AD count: [43], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 3 (F:/Abdipour/Connectivity/ImCoh images\Subject60):
AD count: [29], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 652ms/step
Subject 4 (F:/Abdipour/Connectivity/ImCoh images\Subject14):
AD count: [16], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 476ms/step
Subject 5 (F:/Abdipour/Connectivity/ImCoh images\Subject43):
AD count: [44], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 6 (F:/Abdipour/Connectivity/ImCoh images\Subject29):
AD count: [29], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
Subject 7 (F:/Abdipour/Connectivity/ImCoh images\Subject50):
AD count: [13], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 8 (F:/Abdipour/

## Densenet121


In [ ]:

Dense_base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:

model_name = 'Densenet121_imcoh'
train_and_evaluate_model(Dense_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)

# Pearson

## VGG16

In [23]:
vgg16_base_model = VGG16(weights ='imagenet', include_top =False, input_shape = (224,224,3))

In [24]:
model_name = 'vgg16_pearson'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step - accuracy: 0.9872 - loss: 0.0405
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to F:/Abdipour/Spectrogram Images/\vgg16_pearson_best_weights_iteration_1.weights.h5
236/236 ━━━━━━━━━━━━━━━━━━━━ 107s 446ms/step - accuracy: 0.9872 - loss: 0.0403 - val_accuracy: 1.0000 - val_loss: 4.0210e-21
Epoch 2/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step - accuracy: 1.0000 - loss: 2.7513e-18
Epoch 2: val_accuracy did not improve from 1.00000
236/236 ━━━━━━━━━━━━━━━━━━━━ 108s 457ms/step - accuracy: 1.0000 - loss: 2.7575e-18 - val_accuracy: 1.0000 - val_loss: 4.0210e-21
Epoch 3/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step - accuracy: 1.0000 - loss: 6.3875e-18
Epoch 3: val_accuracy did not improve from 1.00000
236/236 ━━━━━━━━━━━━━━━━━━━━ 107s 455ms/step - accuracy: 1.0000 - loss: 6.4001e-18 - val_accuracy: 1.0000 - val_loss: 4.0210e-21
Epoch 4/100
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step - accuracy: 1.0000 -

f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-package


Predictions for Each Subject's Images:
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 838ms/step
Subject 1 (F:/Abdipour/Connectivity/Pearson images\Subject54):
AD count: [50], HC count: [0]
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 750ms/step
Subject 2 (F:/Abdipour/Connectivity/Pearson images\Subject36):
AD count: [66], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 706ms/step
Subject 3 (F:/Abdipour/Connectivity/Pearson images\Subject13):
AD count: [48], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 892ms/step
Subject 4 (F:/Abdipour/Connectivity/Pearson images\Subject46):
AD count: [51], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step
Subject 5 (F:/Abdipour/Connectivity/Pearson images\Subject57):
AD count: [34], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 903ms/step
Subject 6 (F:/Abdipour/Connectivity/Pearson images\Subject25):
AD count: [19], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 7 (F:/Abdipour/Connectivity/Pearson images\Subject47):
AD count: [23], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 712ms/step


## ResNet50


In [44]:


ResNet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [45]:

model_name = 'ResNet50_pearson'
train_and_evaluate_model(ResNet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)



Iteration 1:
Epoch 1/100
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step - accuracy: 1.0000 - loss: 0.0150

KeyboardInterrupt: 

## Densenet121


In [ ]:



Dense_base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



In [ ]:

model_name = 'Densenet121_pearson'
train_and_evaluate_model(Dense_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)

## InceptionV3

In [ ]:
inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

87910968/87910968 [==============================] - 0s 0us/step


# PSI

## VGG16

In [27]:
vgg16_base_model = VGG16(weights ='imagenet', include_top =False, input_shape = (224,224,3))

In [28]:
model_name = 'vgg16_psi'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
231/231 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step - accuracy: 0.9935 - loss: 0.0457
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to F:/Abdipour/Spectrogram Images/\vgg16_psi_best_weights_iteration_1.weights.h5
231/231 ━━━━━━━━━━━━━━━━━━━━ 110s 470ms/step - accuracy: 0.9935 - loss: 0.0455 - val_accuracy: 1.0000 - val_loss: 2.2694e-19
Epoch 2/100
231/231 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step - accuracy: 1.0000 - loss: 1.0416e-16
Epoch 2: val_accuracy did not improve from 1.00000
231/231 ━━━━━━━━━━━━━━━━━━━━ 107s 465ms/step - accuracy: 1.0000 - loss: 1.0431e-16 - val_accuracy: 1.0000 - val_loss: 2.2694e-19
Epoch 3/100
231/231 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 1.0000 - loss: 6.6954e-17
Epoch 3: val_accuracy did not improve from 1.00000
231/231 ━━━━━━━━━━━━━━━━━━━━ 109s 471ms/step - accuracy: 1.0000 - loss: 6.7147e-17 - val_accuracy: 1.0000 - val_loss: 2.2694e-19
Epoch 4/100
231/231 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step - accuracy: 1.0000 - los

f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-package


Predictions for Each Subject's Images:
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 821ms/step
Subject 1 (F:/Abdipour/Connectivity/PSI images\Subject49):
AD count: [65], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 814ms/step
Subject 2 (F:/Abdipour/Connectivity/PSI images\Subject6):
AD count: [47], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 3 (F:/Abdipour/Connectivity/PSI images\Subject48):
AD count: [28], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 578ms/step
Subject 4 (F:/Abdipour/Connectivity/PSI images\Subject52):
AD count: [43], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 5 (F:/Abdipour/Connectivity/PSI images\Subject35):
AD count: [22], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Subject 6 (F:/Abdipour/Connectivity/PSI images\Subject56):
AD count: [61], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 200ms/step
Subject 7 (F:/Abdipour/Connectivity/PSI images\Subject11):
AD count: [36], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step
Subject 8 (F:/Abdipour/Connectivity

## ResNet50


In [ ]:

ResNet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
model_name = 'ResNet50_psi'
train_and_evaluate_model(ResNet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


## Densenet121


In [ ]:

Dense_base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:

model_name = 'Densenet121_psi'
train_and_evaluate_model(Dense_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)

# AEC

## VGG16

In [34]:
vgg16_base_model = VGG16(weights ='imagenet', include_top =False, input_shape = (224,224,3))

In [35]:
model_name = 'vgg16_AEC'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step - accuracy: 0.9936 - loss: 0.0339
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to F:/Abdipour/Spectrogram Images/\vgg16_AEC_best_weights_iteration_1.weights.h5
235/235 ━━━━━━━━━━━━━━━━━━━━ 115s 483ms/step - accuracy: 0.9936 - loss: 0.0337 - val_accuracy: 1.0000 - val_loss: 2.0875e-25
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - accuracy: 1.0000 - loss: 1.0195e-20
Epoch 2: val_accuracy did not improve from 1.00000
235/235 ━━━━━━━━━━━━━━━━━━━━ 111s 472ms/step - accuracy: 1.0000 - loss: 1.0196e-20 - val_accuracy: 1.0000 - val_loss: 2.0875e-25
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step - accuracy: 1.0000 - loss: 7.6590e-21
Epoch 3: val_accuracy did not improve from 1.00000
235/235 ━━━━━━━━━━━━━━━━━━━━ 109s 463ms/step - accuracy: 1.0000 - loss: 7.6757e-21 - val_accuracy: 1.0000 - val_loss: 2.0875e-25
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step - accuracy: 1.0000 - los

f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-package


Predictions for Each Subject's Images:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 862ms/step
Subject 1 (F:/Abdipour/Connectivity/PSI images\Subject22):
AD count: [19], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 633ms/step
Subject 2 (F:/Abdipour/Connectivity/PSI images\Subject55):
AD count: [46], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 566ms/step
Subject 3 (F:/Abdipour/Connectivity/PSI images\Subject17):
AD count: [44], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 4 (F:/Abdipour/Connectivity/PSI images\Subject3):
AD count: [27], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 567ms/step
Subject 5 (F:/Abdipour/Connectivity/PSI images\Subject43):
AD count: [44], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 198ms/step
Subject 6 (F:/Abdipour/Connectivity/PSI images\Subject5):
AD count: [36], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 7 (F:/Abdipour/Connectivity/PSI images\Subject30):
AD count: [30], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 8 (F:/Abdipour/Connectivity/

## ResNet50


In [36]:
ResNet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [37]:
model_name = 'ResNet50_AEC'
train_and_evaluate_model(ResNet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step - accuracy: 0.9787 - loss: 0.0200
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to F:/Abdipour/Spectrogram Images/\ResNet50_AEC_best_weights_iteration_1.weights.h5
228/228 ━━━━━━━━━━━━━━━━━━━━ 113s 475ms/step - accuracy: 0.9788 - loss: 0.0199 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 2: val_accuracy did not improve from 1.00000
228/228 ━━━━━━━━━━━━━━━━━━━━ 106s 464ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3: val_accuracy did not improve from 1.00000
228/228 ━━━━━━━━━━━━━━━━━━━━ 106s 465ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/100
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step - accuracy: 1.0000 - 

f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Abdipour\pythonProject\venv\lib\site-package


Predictions for Each Subject's Images:
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 602ms/step
Subject 1 (F:/Abdipour/Connectivity/PSI images\Subject63):
AD count: [48], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 594ms/step
Subject 2 (F:/Abdipour/Connectivity/PSI images\Subject7):
AD count: [48], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step
Subject 3 (F:/Abdipour/Connectivity/PSI images\Subject8):
AD count: [20], HC count: [0]
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 633ms/step
Subject 4 (F:/Abdipour/Connectivity/PSI images\Subject36):
AD count: [66], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 950ms/step
Subject 5 (F:/Abdipour/Connectivity/PSI images\Subject21):
AD count: [57], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 6 (F:/Abdipour/Connectivity/PSI images\Subject33):
AD count: [29], HC count: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Subject 7 (F:/Abdipour/Connectivity/PSI images\Subject30):
AD count: [30], HC count: [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step
Subject 8 (F:/Abdipour/Connectivit

## Densenet121


In [39]:
Dense_base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/densenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5: 403 -- Forbidden

In [ ]:
model_name = 'Densenet121_AEC'
train_and_evaluate_model(Dense_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)